In [14]:
import pandas as pd
import csv
from sqlalchemy.orm import sessionmaker
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
# Allow us to declare column types
from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR
from datetime import datetime
from time import time
from numpy import genfromtxt
# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
file_one = "hawaii_measurements.csv"
file_two = "hawaii_stations.csv"

In [3]:
file_one_df = pd.read_csv(file_one)
file_two_df = pd.read_csv(file_two)

In [4]:
file_one_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [5]:
file_one_df.count()

station    19550
date       19550
prcp       18103
tobs       19550
dtype: int64

In [6]:
file_one_df.dtypes

station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [7]:
# Clean the file_one_df for missing rows
file_one_df_cleaned = file_one_df.dropna(how='any')

In [8]:
file_one_df_cleaned.count()

station    18103
date       18103
prcp       18103
tobs       18103
dtype: int64

In [9]:
file_two_df.dtypes

station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object

In [10]:
file_two_df.head(10)

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [11]:
file_two_df.count()

station      9
name         9
latitude     9
longitude    9
elevation    9
dtype: int64

In [12]:
file_one_df_cleaned.to_csv("clean_hawaii_measurements.csv",index=False, header=True)

In [13]:
file_two_df.to_csv("clean_hawaii_stations.csv", index=False, header=True)

In [15]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB using the MySQL Connect Engine
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

In [16]:
# Create Measurement and Station Classes
# ----------------------------------

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Creates Classes which will serve as the anchor points for our Tables
class Station(Base):
    __tablename__ = 'Station'
    id = Column(Integer)
    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


In [17]:
class Measurement(Base):
    __tablename__ = 'Measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)

In [18]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [45]:
file_name = 'clean_hawaii_stations.csv'
df = pd.read_csv(file_name)
df.to_sql(con=engine, index_label='id', name=Station.__tablename__, if_exists='replace')

In [46]:
file_name = 'clean_hawaii_measurements.csv'
df = pd.read_csv(file_name)
df.to_sql(con=engine, index_label='id', name=Measurement.__tablename__, if_exists='replace')

In [48]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [49]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [50]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [51]:
# Print all of the classes mapped to the Base
Base.classes.keys()

[]

In [52]:
inspector = inspect(engine)
inspector.get_table_names()

['Measurement', 'Station']

In [27]:
# Get a list of column names and types
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])
# columns

id INTEGER
station VARCHAR
date DATE
prcp FLOAT
tobs INTEGER


In [28]:
# Get a list of column names and types
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])
# columns

id BIGINT
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [32]:
session = Session(engine)

In [35]:
session.query(func.count(Measurement.date)).all()

[(18103)]

In [36]:
session.query(func.count(Station.name)).all()

[(9)]

In [37]:
engine.dispose()

[]